# AWS CLI Introduction

This is a jupyter notebook that uses the [bash_kernel](https://pypi.org/project/bash_kernel/) to introduce the AWS CLI. Please complete the steps in the [README](./README.md) before running this notebook.

## AWS CLI

The AWS Command Line Interface (AWS CLI) is an open source tool that enables you to interact with AWS services using commands in your command-line shell. See the [documentation](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-welcome.html) for more details.

Amazon Linux hosts used in most labs will have AWS CLI v1 preinstalled. The ACO course and demo video also describe the v1 of the CLI, however development on v1 has now stopped so new services are now only added to v2. v2 is [mostly compatible](https://docs.aws.amazon.com/cli/latest/userguide/cliv2-migration.html) with v1 and has some nice usability improvements, but the [installation](https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html) is a bit more convoluted.

Activity 1 in the ACO course uses a RedHat instance to show students how to install and configure the CLI for their account. If students have their own accounts, or AWS Educate Starter accounts, encourage them to configure the CLI on their desktop to remote access those accounts. You can suggest to your students to use v2 for this activity and for their local installation.

## CLI Installation

Follow the instructions for either [v1](https://docs.aws.amazon.com/cli/latest/userguide/install-cliv1.html) or [v2](https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html).

What version do you have installed?

In [ ]:
aws --version

## Authentication

In a SageMaker notebook your credentials will already be set to the role ypu specified when you created the notebook. For your own account you may need to first run `aws configure` command to set the user access and secret access keys to authenticate with your account.

We can use the AWS Security Token Service to confirm we have properly authenticated to our account. If this returns an error like _"The security token included in the request is invalid."_ you will need to check your authentication.

In [ ]:
aws sts get-caller-identity

If you need to specify you can specify your vocareum lab credentials, you can cut and paste the values here, otherwise skip this step:

In [ ]:
export AWS_ACCESS_KEY_ID=
export AWS_SECRET_ACCESS_KEY=
export AWS_SESSION_TOKEN=
aws sts get-caller-identity

## Help

The aws command format looks like this: 
`aws` `<command>` `<subcommand>` `[options and parameters]`

A `command` could be _ec2_, _s3_, _rds_ or _secretsmanager_, the full list can seen with `aws help`. Possible `subcommand` depend on the `command` selected and these are listed using _help_ as the `subcommand`.

In [ ]:
aws sts help

In [ ]:
aws sts get-caller-identity help

## Output, Query and Filters

Every account in each region always has a default VPC so you can always explore VPCs without have to first create a resource to explore. The [describe-vpcs](https://awscli.amazonaws.com/v2/documentation/api/latest/reference/ec2/describe-vpcs.html) subcommand for `ec2` will output a large JSON blob describing all the VPCs in this account in this region:

In [ ]:
aws ec2 describe-vpcs

We can [query](https://docs.aws.amazon.com/cli/latest/userguide/cli-usage-output.html#cli-usage-output-filter) a subset of the output using [JMESPath](http://jmespath.org/) strings. From the JSON above we see the VPCs are a list, and within the list we can just pull out specific values like the VpcId and CidrBlock. Notice that we can rename those fields if we want to.

In [ ]:
aws ec2 describe-vpcs --query 'Vpcs[*].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault}'

The output format can be changed with `--output` to `table`, `text` (and with v2 `yaml`):

In [ ]:
aws ec2 describe-vpcs --query 'Vpcs[*].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault}' --output table

In [ ]:
aws ec2 describe-vpcs --query 'Vpcs[*].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault}' --output text

In [ ]:
aws ec2 describe-vpcs --query 'Vpcs[*].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault}' --output yaml

If we just want the first VPC:

In [ ]:
aws ec2 describe-vpcs --query 'Vpcs[0].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault}'

Or we can just the default VPC by using the `?` operator. Note the use of backticks in the query expression.

In [ ]:
aws ec2 describe-vpcs --query 'Vpcs[?IsDefault==`true`].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault}'

The preferred way to specify what resources are returned by a command is use the `--filters` option if a filter exists for your resource.

In [ ]:
aws ec2 describe-vpcs --filters 'Name=isDefault,Values=true' --query 'Vpcs[*].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault}'

If you know the ID of resource you are looking for you may be able to specify that as a parameter. Replace `VPCID` with the VpcId from the previous output:

In [ ]:
aws ec2 describe-vpcs --query 'Vpcs[*].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault}' --vpc-ids VPCID